In [4]:
import requests
import json
import pandas as pd
import re
from PIL import Image
import io
import fitz
import PyPDF2
import google.generativeai as genai
from app/utils import filing_list_per_company_number


SyntaxError: invalid syntax (2151460316.py, line 10)

In [2]:
filelist = filing_list_per_company_number('10867753')

filtered = [d for d in filelist if d['description'] == 'incorporation-company']
output_columns = ["Number of Shares", "Type of Shares", "Name"]
output_df = pd.DataFrame(columns=output_columns)
matches = ""

for n in range(len(filtered)):
    incorporation = filtered[n]['links']['document_metadata']
    r = requests.get(incorporation+'/content', auth=(key, ''))
    incorporation_pdf = r.content
# extract text
    pdf_file = io.BytesIO(incorporation_pdf)
    pdf_reader = PyPDF2.PdfReader(pdf_file)
    data = pd.DataFrame(columns=output_columns)
    pdf_document = fitz.open(stream=incorporation_pdf, filetype="pdf")
    for page_num in range(pdf_document.page_count):
        page = pdf_document.load_page(page_num)
        pix = page.get_pixmap(dpi=300)  # Adjust DPI for better quality
        img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)    
        prompt_parts = [img, "Based on the image, create a table, in json output, with the following columns: Number of Shares, Type of Shares, Name. Let Name be the full name of the shareholder. Let the main dictionary be named 'shareholders'. You are looking for the initial shareholders of the company"]
        genai_response = model.generate_content(prompt_parts, generation_config = generation_config)
        genai_response_json = json.loads(genai_response.text)
        data_genai = pd.DataFrame(genai_response_json['shareholders'])
        data = pd.concat([data, data_genai], ignore_index=True, axis=0)
genai_response

# Add document name and description
 #   data['Document Date'] = filtered[n]['date']
 #   data['Document Name'] = filtered[n]['description']
 #   output_df = pd.concat([output_df, data], ignore_index=True, axis=0)

    # Display the DataFrame
#output_df = output_df[output_df["Number of Shares"].notnull() & output_df["Type of Shares"].notnull() & output_df["Name"].notnull()]

#print(output_df)

NameError: name 'filing_list_per_company_number' is not defined

Note: you may need to restart the kernel to use updated packages.
